In [2]:
import pandas as pd
import os
import shutil

In [3]:
os.chdir("..")

dir_root = os.getcwd()

dir_raw = dir_root + u"/sec_filings/Raw"

print(dir_raw)

# Azimuth/Trader/sec_filings/Raw/2019q4_num.txt

/home/wc/Azimuth/Trader/sec_filings/Raw


In [4]:
x = pd.read_csv(dir_raw + u'/2019q4_num.txt', sep='\t')

In [5]:
print(x.head())

adsh                       tag      version         coreg  \
0  0001558370-19-008908  DerivativeNotionalAmount  invest/2013  FerrellgasLP   
1  0001558370-19-008908  DerivativeNotionalAmount  invest/2013  FerrellgasLP   
2  0001558370-19-008908  DerivativeNotionalAmount  invest/2013  FerrellgasLP   
3  0001558370-19-008908  DerivativeNotionalAmount  invest/2013           NaN   
4  0001558370-19-008908  DerivativeNotionalAmount  invest/2013           NaN   

      ddate  qtrs  uom        value footnote  
0  20180531     0  USD  100000000.0      NaN  
1  20170731     0  USD  175000000.0      NaN  
2  20120531     0  USD  140000000.0      NaN  
3  20180531     0  USD  100000000.0      NaN  
4  20180430     0  USD  100000000.0      NaN  


In [6]:
tags = set(x['datatype'].values)
print(tags)

# Trading Symbol

KeyError: 'datatype'

In [0]:
datatypes = ['shares', 'perShare', 'monetary', 'accountingStandard', 'percent', 'tradingSymbol', 'fiscalPeriod', 'decimal']

In [0]:
x = x[x['datatype'].isin(datatypes)]

In [0]:

# Data Source: https://www.sec.gov/dera/data/financial-statement-data-sets.html

# Benjamin Grahmn rules: https://cabotwealth.com/daily/value-investing/benjamin-grahams-value-stock-criteria/ 

##NCAVPS = Current Assets - (Total Liabilities + Preferred Stock) ÷ Shares Outstanding ... less than 1.10 
# PreferredStockValue
# AssetsCurrent 
# Liabilities
# EarningsPerShareBasic
# CommonStockSharesOutstanding

## Debt to Assets = Current Assets / Current Liabilities ... greater than 1.50
# AssetsCurrent  
# LiabilitiesCurrent


## Price / Earnings per Share ratio ... less than 9.0

# EarningsPerShareBasic
# SharePrice

# PRICE TO BOOK VALUE = (P/BV) ... less than 1.20. P/E ratios

# BV = (Total Shareholder Equity−Preferred Stock) / Total Outstanding Shares
​
# SharePrice
# StockholdersEquity 
# PreferredStockValue
# CommonStockSharesOutstanding


# Other rules to cover Benjamin Grahmn: https://www.netnethunter.com/16-benjamin-graham-rules/ 

# Adequate Past Earnings or Catalyst -- I want to invest in firms that have run into major, yet solvable, problems. Companies with a history of barely scraping by could stay valued below NCAV for a very long time. Obviously this doesn't matter if there's a catalyst on the horizon that will either move the stock price or increase profitability.

# Past Price Above NCAV or Catalyst -- This is a good way to tell whether you're buying into a perennial net net or not. I want to see a company that has traded above its NCAV sometime in its recent history, sometime in the past 5 years. The same catalyst clause applies here, as well.

## Warren Buffet Rules 

# ​Debt/Equity is less than 1 and ROE is greater than 10%

# Debt/Equity= Total Liabilities / Total Shareholders’ Equity

# Liabilities
# StockholdersEquity 

# Return on Earnings = (Net Income / Stock Holders Equity)
# NetIncomeLoss
# StockholdersEquity 

#Gross Profit Margin = Gross Profit / Revenue ... is greater than 40% 

# GrossProfit
# SalesRevenueNet
​
# Quarter over Quarter EPS ... is greater than 10
# EarningsPerShareBasic

# https://www.oldschoolvalue.com/tutorial/this-is-how-buffett-interprets-financial-statements/

# Stock Buybacks are greater than last period

# StockRepurchasedAndRetiredDuringPeriodShares


In [0]:
x.to_csv(dir_raw + "/tags_test.csv")